## 今天的作業
在鐵達尼資料集中，以 Titanic_train.csv 中，首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。
* Q1: 目標變數為 'Survived'，以 'Pclass'、'Age'、'SibSp'、'Parch'、'Fare'、'Sex'、'Embarked' 為特徵，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。

In [1]:
# library

import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
import sklearn

# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

## 讀入資料

In [2]:
df_train = pd.read_csv( 'Titanic_train.csv' )
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


把 'Survived' 變成離散型變數 'Survived_cate'

In [4]:
df_train['Survived_cate'] = df_train['Survived'].astype('object')
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived_cate
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


In [5]:
# 檢查訓練資料集中是否有遺失值 ?
def Missing_Counts( Data ) : 
    missing = Data.isnull().sum()  
    missing = missing[ missing>0 ]
    missing.sort_values( inplace=True ) 
    
    Missing_Count = pd.DataFrame( { 'ColumnName':missing.index, 'MissingCount':missing.values } ) 
    Missing_Count[ 'Percentage(%)' ] = Missing_Count['MissingCount'].apply( lambda x:round(x/Data.shape[0]*100,2) )
    return  Missing_Count

Missing_Counts(df_train)

,ColumnName,MissingCount,Percentage(%)
0,Embarked,2,0.22
1,Age,177,19.87
2,Cabin,687,77.10


In [6]:
# 刪除遺失值刪除
complete_data = df_train.dropna()
Missing_Counts(complete_data)

,ColumnName,MissingCount,Percentage(%)


In [7]:
# 刪除每一個人都是獨特的資料，檢視其他變數
complete_data=complete_data.drop( ['Name','Ticket','PassengerId'], axis=1 )
complete_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate
1,1,1,female,38.0,1,0,71.2833,C85,C,1
3,1,1,female,35.0,1,0,53.1000,C123,S,1
6,0,1,male,54.0,0,0,51.8625,E46,S,0
10,1,3,female,4.0,1,1,16.7000,G6,S,1
11,1,1,female,58.0,0,0,26.5500,C103,S,1
...,...,...,...,...,...,...,...,...,...,...
871,1,1,female,47.0,1,1,52.5542,D35,S,1
872,0,1,male,33.0,0,0,5.0000,B51 B53 B55,S,0
879,1,1,female,56.0,0,1,83.1583,C50,C,1
887,1,1,female,19.0,0,0,30.0000,B42,S,1


In [8]:
num_features = []
cat_features = []

for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
    elif dtype == 'object':
        cat_features.append(feature)
    else:
        print( feature, 'is the other type.' )
        
print( f'{len(num_features)} Numeric Features : {num_features}' )
print( f'{len(cat_features)} Category Features : {cat_features}' )

6 Numeric Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
4 Category Features : ['Sex', 'Cabin', 'Embarked', 'Survived_cate']


## 定義 x,y

In [9]:
x = complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']]
y = complete_data['Survived']

In [10]:
x

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S
...,...,...,...,...,...,...,...
871,1,47.0,1,1,52.5542,female,S
872,1,33.0,0,0,5.0000,male,S
879,1,56.0,0,1,83.1583,female,C
887,1,19.0,0,0,30.0000,female,S


In [11]:
y

1      1
3      1
6      0
10     1
11     1
      ..
871    1
872    0
879    1
887    1
889    1
Name: Survived, Length: 183, dtype: int64

### Q1: 目標變數為 'Survived'，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性!

### 包裝法(Wrapper Method)

In [12]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

#### Step 1: 離散型態的欄位轉成數值型態

In [13]:
# 將 'Sex'、'Embarked' 欄位的型態，從離散轉換成數值。

x = x.copy()

for col in ['Sex','Embarked']:
    mapping = {}
    category = x.loc[:,col].astype('category')
    mapping = dict(enumerate(category.cat.categories))
    print( col,' 欄位的型態轉換: ', mapping )
    x.loc[:,col] = x.loc[:,col].astype('category').cat.codes 

Sex  欄位的型態轉換:  {0: 'female', 1: 'male'}
Embarked  欄位的型態轉換:  {0: 'C', 1: 'Q', 2: 'S'}


In [14]:
x

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,0,0
3,1,35.0,1,0,53.1000,0,2
6,1,54.0,0,0,51.8625,1,2
10,3,4.0,1,1,16.7000,0,2
11,1,58.0,0,0,26.5500,0,2
...,...,...,...,...,...,...,...
871,1,47.0,1,1,52.5542,0,2
872,1,33.0,0,0,5.0000,1,2
879,1,56.0,0,1,83.1583,0,0
887,1,19.0,0,0,30.0000,0,2


#### Step 2: 使用 SVC(Support Vector Classification) 方法

In [15]:
estimator = SVC( kernel='linear' )
selector = RFE( estimator, n_features_to_select=1, step=1 )
selector = selector.fit( x, y )

## If True, this feature is selected.
print( selector.support_ )

## Selected (i.e., estimated best) features are assigned rank 1.
ranking = selector.ranking_
print( ranking )

rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

[False False False False False  True False]
[3 5 2 6 7 1 4]
['Sex']
